In [1]:
!pip install sentence_transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [2]:
import csv
import random

from sentence_transformers.readers import InputExample

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:

def load_train_kor_sts(filename):
    samples = []
    with open(filename, "rt", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
        for row in reader:
            score = float(row["score"]) / 5.0
            samples.append(InputExample(texts=[row["sentence1"], row["sentence2"]], label=score))
    return samples


def load_train_kor_nli(filename):
    data = {}

    def add_sampling(samplingA, samplingB, label):
        if  samplingA not in data:
            data[samplingA] = {"contradiction": set(), "entailment": set(), "neutral": set()}
        data[samplingA][label].add(samplingB)

    with open(filename, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f, delimiter="\t", quoting=csv.QUOTE_NONE)
            for row in reader:
                samplingA = row["sentence1"].strip()
                samplingB = row["sentence2"].strip()
                add_sampling(samplingA, samplingB, row["gold_label"])
                add_sampling(samplingB, samplingA, row["gold_label"])

    samples = []

    for sampling, etc in data.items():
            if len(etc["entailment"]) > 0 and len(etc["contradiction"]) > 0:
                samples.append(
                    InputExample(
                        texts=[
                            sampling,
                            random.choice(list(etc["entailment"])),
                            random.choice(list(etc["contradiction"])),
                        ]
                    )
                )
                samples.append(
                    InputExample(
                        texts=[
                            random.choice(list(etc["entailment"])),
                            sampling,
                            random.choice(list(etc["contradiction"])),
                        ]
                    )
                )
    return samples

In [4]:
import glob
import logging
import math
import os
import random
from datetime import datetime

import numpy as np
import torch
from sentence_transformers import LoggingHandler, SentenceTransformer, datasets, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)

# Modify as needed
model_path = "Alibaba-NLP/gte-multilingual-base"
max_seq_length = 256
nli_batch_size = 64
sts_batch_size = 8
num_epochs = 10
eval_steps = 1000
learning_rate = 2e-5
seed = 500

def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

In [5]:
set_seeds(seed)

model_save_path = os.path.join("output/gte-kor-turbo" + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Use trust_remote_code=True when loading the SentenceTransformer model
model = SentenceTransformer(model_path, trust_remote_code=True)

# Use the model to set up the rest of the training pipeline
base_model = model._first_module()
pooling_model = models.Pooling(base_model.get_word_embedding_dimension(), pooling_mode="mean")
model = SentenceTransformer(modules=[base_model, pooling_model])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/123k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
nli_load_path = "dataset/KorNLI"
sts_load_path = "dataset/KorSTS"

logging.info("Read KorNLI, STS dataset")

nli_train_datasets = glob.glob(os.path.join(nli_load_path, "*train.ko.tsv"))
dev_sts_path = os.path.join(sts_load_path, "sts-dev.tsv")

In [10]:
nli_train_samples = []
for nli_train_data in nli_train_datasets:
        nli_train_samples += load_train_kor_nli(nli_train_data)

nli_train_dataloader = datasets.NoDuplicatesDataLoader(nli_train_samples, batch_size=nli_batch_size)
nli_train_loss = losses.MultipleNegativesRankingLoss(model)

sts_dataset_path = "dataset/KorSTS"
sts_train_file = os.path.join(sts_dataset_path, "sts-train.tsv")

sts_train_samples = load_train_kor_sts(sts_train_file)
sts_train_dataloader = DataLoader(sts_train_samples, shuffle=True, batch_size=sts_batch_size)
sts_train_loss = losses.CosineSimilarityLoss(model=model)

dev_samples = load_train_kor_sts(dev_sts_path)
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=sts_batch_size, name="sts-dev")

print("Length of NLI data loader:", len(nli_train_dataloader))
print("Length of STS data loader:", len(sts_train_dataloader))
steps_per_epoch = min(len(nli_train_dataloader), len(sts_train_dataloader))

epoch_steps = math.ceil(steps_per_epoch * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("epoch-steps: {}".format(epoch_steps))


Length of NLI data loader: 8885
Length of STS data loader: 719


In [11]:
train_objectives = [(nli_train_dataloader, nli_train_loss), (sts_train_dataloader, sts_train_loss)]
model.fit(
        train_objectives=train_objectives,
        evaluator=dev_evaluator,
        epochs=num_epochs,
        optimizer_params={"lr": learning_rate},
        evaluation_steps=eval_steps,
        warmup_steps=epoch_steps,
        output_path=model_save_path,
    )

    # Load the trained model with trust_remote_code=True
model = SentenceTransformer(model_save_path, trust_remote_code=True)
logging.info("Start benchmark test dataset")

test_file = os.path.join(sts_dataset_path, "sts-test.tsv")
test_samples = load_train_kor_sts(test_file)

test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name="sts-test")
test_evaluator(model, output_path=model_save_path)

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine,Sts-dev Pearson Manhattan,Sts-dev Spearman Manhattan,Sts-dev Pearson Euclidean,Sts-dev Spearman Euclidean,Sts-dev Pearson Dot,Sts-dev Spearman Dot,Sts-dev Pearson Max,Sts-dev Spearman Max
1000,0.229000,No log,0.858991,0.858775,0.824528,0.828948,0.826413,0.830631,0.768883,0.791405,0.858991,0.858775
1439,0.229000,No log,0.861028,0.862134,0.805702,0.808820,0.807666,0.810938,0.775075,0.793663,0.861028,0.862134
2000,0.139300,No log,0.858028,0.858038,0.807258,0.813699,0.809013,0.815446,0.749780,0.778043,0.858028,0.858038
2878,0.082100,No log,0.854626,0.855881,0.803050,0.808008,0.804047,0.808568,0.753913,0.778910,0.854626,0.855881
3000,0.084300,No log,0.853576,0.853807,0.803296,0.807991,0.804571,0.809131,0.754921,0.779587,0.853576,0.853807
4000,0.034900,No log,0.855968,0.855753,0.796958,0.803958,0.797890,0.804879,0.742730,0.773791,0.855968,0.855753
4317,0.034900,No log,0.855507,0.856369,0.795194,0.802185,0.795821,0.802352,0.743280,0.777752,0.855507,0.856369
5000,0.019500,No log,0.852333,0.852684,0.789693,0.797202,0.790131,0.797484,0.745418,0.778541,0.852333,0.852684
5756,0.018100,No log,0.851779,0.852340,0.783175,0.789252,0.783898,0.790026,0.727303,0.765721,0.851779,0.852340
6000,0.017400,No log,0.854313,0.854520,0.789086,0.795694,0.789951,0.796385,0.742920,0.777523,0.854313,0.854520


Computing widget examples:   0%|          | 0/2 [00:00<?, ?example/s]

{'sts-test_pearson_cosine': 0.8069414831172336,
 'sts-test_spearman_cosine': 0.8093097331207868,
 'sts-test_pearson_manhattan': 0.7249630747888534,
 'sts-test_spearman_manhattan': 0.7326850346990167,
 'sts-test_pearson_euclidean': 0.727283972664176,
 'sts-test_spearman_euclidean': 0.734226546536339,
 'sts-test_pearson_dot': 0.5790210756163109,
 'sts-test_spearman_dot': 0.6552036518432575,
 'sts-test_pearson_max': 0.8069414831172336,
 'sts-test_spearman_max': 0.8093097331207868}